In [1]:
import pandas as pd

df = pd.read_csv('stockmarket.csv')

df.head()

,Date,Open,High,Low,Close,Volume
0,2003-01-20,41.55,41.99,41.39,41.39,2454991.0
1,2003-01-21,41.70,42.02,40.33,40.40,3279902.0
2,2003-01-22,40.40,40.95,38.32,38.75,4568929.0
3,2003-01-23,38.89,39.50,38.32,38.70,3879946.0
4,2003-01-24,38.77,38.92,36.66,36.93,5196923.0


In [2]:
df_new = pd.DataFrame()

df_new['open'] = df['Open']
df_new['date'] = df['Date']

df_new['open_1'] = df['Open'].shift(1)
df_new['close_1'] = df['Close'].shift(1)
df_new['high_1'] = df['High'].shift(1)
df_new['low_1'] = df['Low'].shift(1)
df_new['volume_1'] = df['Volume'].shift(1)

In [3]:
df_new['avg_price_7'] = df['Close'].rolling(window = 5).mean().shift(1)
df_new['avg_price_30'] = df['Close'].rolling(window = 20).mean().shift(1)
df_new['avg_price_365'] = df['Close'].rolling(window = 240).mean().shift(1)

In [4]:
df_new['avg_volume_5'] = df['Close'].rolling(window = 5).mean().shift(1)
df_new['avg_volume_30'] = df['Close'].rolling(window = 20).mean().shift(1)
df_new['avg_volume_365'] = df['Close'].rolling(window = 240).mean().shift(1)

In [5]:
df_new['ratio_avg_volume_5_30'] = df_new['avg_volume_5'] / df_new['avg_volume_30']
df_new['ratio_avg_volume_5_365'] = df_new['avg_volume_5'] / df_new['avg_volume_30']
df_new['ratio_avg_volume_30_365'] = df_new['avg_volume_30'] / df_new['avg_volume_365']

In [6]:
df_new['std_price_5'] = df['Close'].rolling(window = 5).std().shift(1)
df_new['std_price_30'] = df['Close'].rolling(window = 20).std().shift(1)
df_new['std_price_365'] = df['Close'].rolling(window = 240).std().shift(1)

In [7]:
df_new['ratio_std_price_5_30'] = df_new['std_price_5'] / df_new['std_price_30']
df_new['ratio_std_price_5_365'] = df_new['std_price_5'] / df_new['std_price_365']
df_new['ratio_std_price_30_365'] = df_new['std_price_30'] / df_new['std_price_365']

In [8]:
df_new['std_volume_5'] = df['Close'].rolling(window = 5).std().shift(1)
df_new['std_volume_30'] = df['Close'].rolling(window = 20).std().shift(1)
df_new['std_volume_365'] = df['Close'].rolling(window = 240).std().shift(1)

In [9]:
df_new['ratio_std_volume_5_30'] = df_new['std_volume_5'] / df_new['std_volume_30']
df_new['ratio_std_volume_5_365'] = df_new['std_volume_5'] / df_new['std_volume_365']
df_new['ratio_std_volume_30_365'] = df_new['std_volume_30'] / df_new['std_volume_365']

In [10]:
df_new['return_1']=((df['Close'].shift(1)) / df['Close'].shift(1)).shift(1)
df_new['return_5']=((df['Close'].shift(5)) / df['Close'].shift(5)).shift(1)
df_new['return_30']=((df['Close'].shift(20)) / df['Close'].shift(20)).shift(1)
df_new['return_30']=((df['Close'].shift(240)) / df['Close'].shift(240)).shift(1)

In [11]:
df_new['moving_avg_5'] = df_new['return_1'].rolling(window = 5).mean().shift(1)
df_new['moving_avg_30'] = df_new['return_1'].rolling(window = 5).mean().shift(20)
df_new['moving_avg_365'] = df_new['return_1'].rolling(window = 5).mean().shift(240)

In [12]:
df_new['close'] = df['Close']
df_new['date'] = df['Date']
df_new = df_new.dropna(axis= 0)

In [13]:
train_data = df_new[df_new['date'] <= '2016-12-31']
test_data = df_new[df_new['date'] <= '2017-01-01']

In [14]:
x_train_data = train_data.drop(['close', 'date'], axis=1)
y_train_data = train_data['close']

x_test_data = test_data.drop(['close', 'date'], axis=1)
y_test_data = test_data['close']

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_scaled_train = scaler.fit_transform(x_train_data)
x_scaled_test = scaler.transform(x_test_data)

In [16]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV

param_grid = {'alpha' : [3e-06, 1e-5, 3e-5], 'eta0' : [0.01, 0.03, 0.1]}

lr = SGDRegressor(penalty='l2')

grid_search = GridSearchCV(lr, param_grid, cv=5, scoring= 'neg_mean_absolute_error')

grid_search.fit(x_scaled_train, y_train_data)

print(grid_search.best_params_)

{'alpha': 1e-05, 'eta0': 0.01}


In [17]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

lr_best = grid_search.best_estimator_

predictions = lr_best.predict(x_scaled_test)

print('MSE: {}'.format(mean_squared_error(y_test_data,predictions)))
print('MAE:{}'.format(mean_absolute_error(y_test_data,predictions)))
print('R squared:{}'.format(r2_score(y_test_data,predictions)))

MSE: 1.681664142871937
MAE:0.5186458365354135
R squared:0.9988925242025897
